# This is a web scraping demo, for:


*   extracting all images from an article (url)
*   filtering them (e.g. ignoring small icons, duplicates etc.)
* testing bulk requests to the external Hive API
* aggregating the results into a "trustworthiness score" for an article/ post etc.



##Environment Config

In [1]:
!pip install -q selenium
!pip install -q webdriver-manager
!pip install -q google-colab-selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 18.0 MB/s eta 0:00:00


In [67]:
import requests
import re
import time
import google_colab_selenium as gcs

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
from urllib.parse import urljoin
from webdriver_manager.chrome import ChromeDriverManager
from concurrent.futures import ThreadPoolExecutor

In [3]:
from google.colab import userdata
API_KEY = userdata.get('API_KEY')

##Extract all images

###basic version

In [91]:
def get_images_from_page_basic(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return []

    soup = BeautifulSoup(response.text, "html.parser")

    img_tags = soup.find_all("img")

    img_urls = [img.get("src") for img in img_tags if img.get("src")]
    img_urls = [urljoin(url, img_url) for img_url in img_urls]

    return img_urls

In [92]:
page_url_1 = "https://www.bbc.com/news/articles/c5yv5976z9po"
page_url_2 = "https://www.spectator.co.uk/article/is-this-new-chinese-ai-even-better-than-deepseek/"
page_url_3 = "https://www.theverge.com/24353060/deepseek-ai-china-nvidia-openai"
page_url_4 = "https://www.bu.edu/articles/2025/does-chinas-deepseek-represent-a-new-frontier-in-ai/"

test_urls = [page_url_1, page_url_2, page_url_3, page_url_4]

for url in test_urls:
  images = get_images_from_page_basic(url)
  print_found_imgs(images)
  print(" --- ")

Found 2 images:
https://www.bbc.com/bbcx/grey-placeholder.png
https://ichef.bbci.co.uk/news/480/cpsprodpb/3cb2/live/61ec64f0-dd4c-11ef-b20c-cf1b3bd7a488.jpg.webp
 --- 
Found 15 images:
https://www.spectator.co.uk/wp-content/uploads/2025/03/cover-15032025-issue.jpg?w=358
https://www.spectator.co.uk/wp-content/uploads/2025/03/cover-15032025-issue.jpg?w=358
https://www.spectator.co.uk/wp-content/uploads/2025/03/iStock-1456167226.jpg?w=1280
https://www.spectator.co.uk/wp-content/uploads/2025/03/iStock-1456167226.jpg?w=1365
https://src.spectator.co.uk/wp-content/uploads/2023/01/Newsletter-plane_-footer_1000w.jpg
https://www.spectator.co.uk/wp-content/uploads/2025/03/iStock-1456167226.jpg?w=1365
https://www.spectator.co.uk/wp-content/uploads/2025/03/GettyImages-1246845745-2.jpg?resize=378,213
https://www.spectator.co.uk/wp-content/uploads/2025/03/POLCOL.png?resize=200,133
https://www.spectator.co.uk/wp-content/uploads/2025/01/PA-78818805.jpg?resize=200,133
https://www.spectator.co.uk/wp-cont

In [28]:
page_url = "https://www.theverge.com/24353060/deepseek-ai-china-nvidia-openai"

image_urls = get_images_from_page_basic(page_url)

print(f"Found {len(image_urls)} images:")
for img_url in image_urls:
    print(img_url)

Found 52 images:
https://www.google-analytics.com/g/collect?v=2&tid=G-C3QZPB4GVE&cid=555&en=noscript_page_view
https://platform.theverge.com/wp-content/uploads/sites/2/chorus/author_profile_images/195810/EMMA_ROTH.0.jpg?quality=90&strip=all&crop=0%2C0%2C100%2C100&w=2400
https://platform.theverge.com/wp-content/uploads/sites/2/chorus/author_profile_images/197794/dominic-preston.0.jpg?quality=90&strip=all&crop=0%2C0%2C100%2C100&w=2400
https://platform.theverge.com/wp-content/uploads/sites/2/2025/01/0007835275_20.jpg?quality=90&strip=all&crop=0%2C0%2C100%2C100&w=2400
https://platform.theverge.com/wp-content/uploads/sites/2/chorus/author_profile_images/197794/dominic-preston.0.jpg?quality=90&strip=all&crop=0%2C0%2C100%2C100&w=2400
https://platform.theverge.com/wp-content/uploads/sites/2/chorus/author_profile_images/195819/JAY_PETERS.0.jpg?quality=90&strip=all&crop=0%2C0%2C100%2C100&w=2400
https://platform.theverge.com/wp-content/uploads/sites/2/chorus/uploads/chorus_asset/file/25728971/STK

###more advanced version

In [ ]:
def click_element(driver, xpath, timeout=3):
  try:
    element = WebDriverWait(driver, timeout).until(EC.element_to_be_clickable((By.XPATH, xpath)))
    print(f"Clicking {xpath}")
    element.click()
    time.sleep(0.5)
    return True
  except Exception:
    return False



---



In [51]:
def set_options():
  options = Options();
  options.add_argument("--headless=new")
  options.add_argument("--no-sandbox")
  options.add_argument("--disable-dev-shm-usage")
  options.add_argument("--disable-blink-features=AutomationControlled")
  options.binary_location = "/usr/bin/google-chrome"
  return options

In [5]:
def scroll_page(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [94]:
js_code = """
        const exclusionPattern = /avatar|loading|icon|thumbnail|logo|banner|footer/i;

        function isValidImage(img, minSize) {
            let src = img.getAttribute('src');
            let srcset = img.getAttribute('srcset');
            let width = img.getAttribute('width');
            let height = img.getAttribute('height');
            let className = img.getAttribute('class') || "";
            let alt = img.getAttribute('alt') || "";
            let style = img.getAttribute('style') || "";

            // Fallback to srcset if src is absent or invalid
            if (!src || !src.startsWith('http')) {
                if (srcset) {
                    let srcsetUrls = srcset.split(',')
                        .map(s => s.trim().split(' ')[0])
                        .filter(url => url && url.startsWith('http'));
                    src = srcsetUrls.length > 0 ? srcsetUrls[0] : null;
                }
            }
            if (!src) return null;
            if ((width && width < minSize) || (height && height < minSize)) return null;
            if (style.includes("display: none") || style.includes("opacity: 0")) return null;
            if (exclusionPattern.test(src)) return null;
            if (exclusionPattern.test(className)) return null;
            if (exclusionPattern.test(alt)) return null;
            return src;
        }

        let imgSet = new Set();
        document.querySelectorAll('img').forEach(img => {
            let validSrc = isValidImage(img, arguments[0]);
            if (validSrc) {
                imgSet.add(validSrc);
            }
        });
        return Array.from(imgSet);
        """

In [95]:
def get_images_from_page(url, min_size=100):
    options = set_options()
    driver = gcs.Chrome(options=options)

    try:
        #print("Opening URL: ", url)
        driver.get(url)

        #print("Waiting for document to load")
        WebDriverWait(driver, 10).until(
            lambda d: d.execute_script("return document.readyState") == "complete"
        )

        #print("Removing pop-ups")
        driver.execute_script("document.querySelectorAll('.popup-class').forEach(e => e.remove());")

        #print("Scrolling the page")
        scroll_page(driver)

        #print("Extracting image URLs")
        img_urls = driver.execute_script(js_code, min_size)

    except Exception as e:
        print("An error occurred: %s", e)
        raise
    finally:
        driver.quit()

    return img_urls


Filter rules:


*   skip **small** images (less than 100)
* skip classes/ alt containing: **avatar, loading, icon, thumbnail, logo, banner, footer**
* skip **invisible** images



In [82]:
def print_found_imgs(images):
  print(f"Found {len(images)} image(s):")
  for img in images:
      print(img)

In [96]:
page_url_1 = "https://www.bbc.com/news/articles/c5yv5976z9po"
page_url_2 = "https://www.spectator.co.uk/article/is-this-new-chinese-ai-even-better-than-deepseek/"
page_url_3 = "https://www.theverge.com/24353060/deepseek-ai-china-nvidia-openai"
page_url_4 = "https://www.bu.edu/articles/2025/does-chinas-deepseek-represent-a-new-frontier-in-ai/"

test_urls = [page_url_1, page_url_2, page_url_3, page_url_4]

for url in test_urls:
  images = get_images_from_page(url)
  print_found_imgs(images)
  print(" --- ")

<IPython.core.display.Javascript object>

Found 1 images:
https://ichef.bbci.co.uk/news/480/cpsprodpb/3cb2/live/61ec64f0-dd4c-11ef-b20c-cf1b3bd7a488.jpg.webp
 --- 


<IPython.core.display.Javascript object>

Found 2 images:
https://www.spectator.co.uk/wp-content/uploads/2025/03/cover-15032025-issue.jpg?w=358
https://www.spectator.co.uk/wp-content/uploads/2025/03/iStock-1456167226.jpg?w=1365
 --- 


<IPython.core.display.Javascript object>

Found 10 images:
https://platform.theverge.com/wp-content/uploads/sites/2/2025/01/STKB320_DEEPSEEK_AI_CVIRGINIA_B.jpg?quality=90&strip=all&crop=0%2C0%2C100%2C100&w=2400
https://platform.theverge.com/wp-content/uploads/sites/2/2025/01/VST_0131_Site.jpg?quality=90&strip=all&crop=0%2C0%2C100%2C100&w=2400
https://platform.theverge.com/wp-content/uploads/sites/2/2025/01/247141_NOTEPAD_DEEPSEEK_AI_MICROSOFT_CVIRGINIA-1.jpg?quality=90&strip=all&crop=0%2C0%2C100%2C100&w=2400
https://platform.theverge.com/wp-content/uploads/sites/2/2025/01/DCD_013024_v1.jpg?quality=90&strip=all&crop=0%2C0%2C100%2C100&w=2400
https://platform.theverge.com/wp-content/uploads/sites/2/chorus/uploads/chorus_asset/file/25546251/STK169_Mark_Zuckerburg_CVIRGINIA_C.jpg?quality=90&strip=all&crop=0%2C0%2C100%2C100&w=2400
https://platform.theverge.com/wp-content/uploads/sites/2/2025/01/STKB320_DEEPSEEK_AI_CVIRGINIA_D.jpg?quality=90&strip=all&crop=0%2C0%2C100%2C100&w=2400
https://platform.theverge.com/wp-content/uploads/site

<IPython.core.display.Javascript object>

Found 1 images:
https://www.bu.edu/files/2022/08/resize-22-1553-MARCOMHEAD-161-3-500x500.jpg
 --- 
